In [1]:
import json
import os
import numpy as np
import pandas as pd
import plotnine
from plotnine import *  # Provides a ggplot-like interface to matplotlib.

# Get the BigQuery curated dataset for the current workspace context.
CDR = os.environ['WORKSPACE_CDR']

# Get workspace CDR version
version = %env WORKSPACE_CDR

# =============================================================================
# APPROACH 1: Direct SNOMED Search
# Search for RA concepts directly in SNOMED vocabulary by keyword
# This approach may capture concepts not mapped from ICD codes
# =============================================================================

ra_snomed_df = pd.read_gbq(f"""
SELECT DISTINCT
  c.concept_id,
  c.concept_name,
  c.concept_code,
  c.vocabulary_id,
  c.domain_id,
  c.standard_concept
FROM `{version}.concept` AS c
WHERE c.domain_id = 'Condition'
  AND c.vocabulary_id = 'SNOMED'
  AND c.standard_concept = 'S'
  AND LOWER(c.concept_name) LIKE '%rheumatoid arthritis%'
""", dialect="standard")

# Result: 99 SNOMED concepts containing "rheumatoid arthritis"
ra_snomed_df.shape

# Export concept IDs for use in cohort building
concept_list = ra_snomed_df[['concept_id']]
concept_list.to_csv("ra_concept_list.csv", index=False)

# =============================================================================
# APPROACH 2: PheCode-to-OMOP Mapping
# Map PheCode 714.1 (RA) → ICD-9/ICD-10 → OMOP concept IDs
# This ensures alignment with established phenotyping definitions
# =============================================================================

# -----------------------------------------------------------------------------
# RHEUMATOID ARTHRITIS (PheCode 714.1)
# -----------------------------------------------------------------------------

# Step 1: Load PheCode-to-ICD mapping file
phe_file = "Phecode714_1_RA.csv"
phe = pd.read_csv(phe_file, dtype=str)

# Step 2: Separate ICD-9 and ICD-10 codes
icd9 = sorted(phe[phe["Flag"] == "9"]["ICD"].unique())
icd10 = sorted(phe[phe["Flag"] == "10"]["ICD"].unique())

# Format for SQL IN clause
icd9_str = ", ".join(f"'{c}'" for c in icd9)
icd10_str = ", ".join(f"'{c}'" for c in icd10)

# Step 3: Query OMOP concept table for ICD source concepts
ra_concepts_df = pd.read_gbq(f"""
SELECT DISTINCT
  c.concept_id,
  c.concept_name,
  c.concept_code,
  c.vocabulary_id,
  c.domain_id,
  c.standard_concept
FROM `{version}.concept` AS c
WHERE (c.vocabulary_id = 'ICD9CM'  AND c.concept_code IN ({icd9_str}))
   OR (c.vocabulary_id = 'ICD10CM' AND c.concept_code IN ({icd10_str}))
""", dialect="standard")

# Result: 410 ICD source concepts (non-standard)
ra_concepts_df.shape

# Step 4: Map ICD codes to standard SNOMED concepts via concept_relationship
ra_snomed_df = pd.read_gbq(f"""
WITH src AS (
  -- Get all ICD concept IDs for this PheCode
  SELECT DISTINCT concept_id
  FROM `{version}.concept`
  WHERE (vocabulary_id = 'ICD9CM'  AND concept_code IN ({icd9_str}))
     OR (vocabulary_id = 'ICD10CM' AND concept_code IN ({icd10_str}))
)
SELECT DISTINCT
  s.concept_id,
  s.concept_name,
  s.concept_code,
  s.vocabulary_id
FROM src
JOIN `{version}.concept_relationship` r
  ON src.concept_id = r.concept_id_1
 AND r.relationship_id = 'Maps to'  -- Standard mapping relationship
JOIN `{version}.concept` s
  ON r.concept_id_2 = s.concept_id
WHERE s.standard_concept = 'S'      -- Standard concepts only
  AND s.domain_id = 'Condition'
""", dialect="standard")

# Result: 48 standard SNOMED concepts
ra_snomed_df.shape

# Export for downstream analysis
ra_snomed_df['concept_id'].to_csv("RA_SNOMED_Concepts.csv", index=False)

# -----------------------------------------------------------------------------
# DIABETES (PheCode 250)
# -----------------------------------------------------------------------------

# Load PheCode mapping and separate ICD versions
phe_file = "Phecode250_diabetes.csv"
phe = pd.read_csv(phe_file, dtype=str)

icd9 = sorted(phe[phe["Flag"] == "9"]["ICD"].unique())
icd10 = sorted(phe[phe["Flag"] == "10"]["ICD"].unique())

icd9_str = ", ".join(f"'{c}'" for c in icd9)
icd10_str = ", ".join(f"'{c}'" for c in icd10)

# Query ICD source concepts
diabetes_concepts_df = pd.read_gbq(f"""
SELECT DISTINCT
  c.concept_id,
  c.concept_name,
  c.concept_code,
  c.vocabulary_id,
  c.domain_id,
  c.standard_concept
FROM `{version}.concept` AS c
WHERE (c.vocabulary_id = 'ICD9CM'  AND c.concept_code IN ({icd9_str}))
   OR (c.vocabulary_id = 'ICD10CM' AND c.concept_code IN ({icd10_str}))
""", dialect="standard")

# Result: 578 ICD source concepts
diabetes_concepts_df.shape

# Map to standard SNOMED concepts
diabetes_snomed_df = pd.read_gbq(f"""
WITH src AS (
  SELECT DISTINCT concept_id
  FROM `{version}.concept`
  WHERE (vocabulary_id = 'ICD9CM'  AND concept_code IN ({icd9_str}))
     OR (vocabulary_id = 'ICD10CM' AND concept_code IN ({icd10_str}))
)
SELECT DISTINCT
  s.concept_id,
  s.concept_name,
  s.concept_code,
  s.vocabulary_id
FROM src
JOIN `{version}.concept_relationship` r
  ON src.concept_id = r.concept_id_1
 AND r.relationship_id = 'Maps to'
JOIN `{version}.concept` s
  ON r.concept_id_2 = s.concept_id
WHERE s.standard_concept = 'S'
  AND s.domain_id = 'Condition'
""", dialect="standard")

# Result: 136 standard SNOMED concepts
diabetes_snomed_df.shape

diabetes_snomed_df['concept_id'].to_csv("Diabetes_SNOMED_Concepts.csv", index=False)

# -----------------------------------------------------------------------------
# HYPERTENSION (PheCode 401)
# -----------------------------------------------------------------------------

# Load PheCode mapping and separate ICD versions
phe_file = "Phecode401_hypertension.csv"
phe = pd.read_csv(phe_file, dtype=str)

icd9 = sorted(phe[phe["Flag"] == "9"]["ICD"].unique())
icd10 = sorted(phe[phe["Flag"] == "10"]["ICD"].unique())

icd9_str = ", ".join(f"'{c}'" for c in icd9)
icd10_str = ", ".join(f"'{c}'" for c in icd10)

# Query ICD source concepts
hypertension_concepts_df = pd.read_gbq(f"""
SELECT DISTINCT
  c.concept_id,
  c.concept_name,
  c.concept_code,
  c.vocabulary_id,
  c.domain_id,
  c.standard_concept
FROM `{version}.concept` AS c
WHERE (c.vocabulary_id = 'ICD9CM'  AND c.concept_code IN ({icd9_str}))
   OR (c.vocabulary_id = 'ICD10CM' AND c.concept_code IN ({icd10_str}))
""", dialect="standard")

# Result: 78 ICD source concepts
hypertension_concepts_df.shape

# Map to standard SNOMED concepts
hypertension_snomed_df = pd.read_gbq(f"""
WITH src AS (
  SELECT DISTINCT concept_id
  FROM `{version}.concept`
  WHERE (vocabulary_id = 'ICD9CM'  AND concept_code IN ({icd9_str}))
     OR (vocabulary_id = 'ICD10CM' AND concept_code IN ({icd10_str}))
)
SELECT DISTINCT
  s.concept_id,
  s.concept_name,
  s.concept_code,
  s.vocabulary_id
FROM src
JOIN `{version}.concept_relationship` r
  ON src.concept_id = r.concept_id_1
 AND r.relationship_id = 'Maps to'
JOIN `{version}.concept` s
  ON r.concept_id_2 = s.concept_id
WHERE s.standard_concept = 'S'
  AND s.domain_id = 'Condition'
""", dialect="standard")

# Result: 47 standard SNOMED concepts
hypertension_snomed_df.shape

hypertension_snomed_df['concept_id'].to_csv("Hypertension_SNOMED_Concepts.csv", index=False)